In [1]:
from B1Py.calibration import KalibrExtractExtrinsics
import numpy as np

In [2]:
import rclpy
rclpy.init()

In [3]:
ext = KalibrExtractExtrinsics('/home/rstaion/Downloads/rs455i_142422250881_kalibr-camchain.yaml')
ext.keys()

dict_keys(['cam0', 'cam0_T_imu', 'cam1', 'cam1_T_imu', 'cam1_T_cam0', 'cam2', 'cam2_T_imu', 'cam2_T_cam1'])

In [4]:
oir1_T_ocolor = np.linalg.inv(ext['cam1_T_cam0'])@np.linalg.inv(ext['cam2_T_cam1'])
oir1_T_oir2 = np.linalg.inv(ext['cam1_T_cam0'])
oir1_T_imu = np.linalg.inv(ext['cam0_T_imu'])

In [5]:
from B1Py.calibration import ExtrinsicCalibrationManager
manager = ExtrinsicCalibrationManager()

In [6]:
for key, val in ext.items():
    if '_T_' in key:
        parent = key.split('_T_')[0]
        child = key.split('_T_')[-1]
        rot = val[:3, :3]
        trans = val[:3, 3]
        manager.add(parent, child, rot, trans)

In [7]:
oir1_T_ocolor = manager.get('cam0', 'cam2')
oir1_T_oir2 = manager.get('cam0', 'cam1')
oir1_T_imu = manager.get('cam0', 'imu')

In [8]:
from rclpy.node import Node
import tf2_ros
from geometry_msgs.msg import TransformStamped
from scipy.spatial.transform import Rotation as R
import numpy as np  
from B1Py.robot.interface.ros2 import ROS2ExecutorManager
class RealsenseExtrinsicsBroadcaster(Node):

    def __init__(self, camera_name,  oir1_T_oir2, oir1_T_ocolor, oir1_T_imu=None):
        super().__init__(f'{camera_name}_tf2_broadcaster')

        self.body_frame_name = f'{camera_name}_link'
        self.ir1_frame_name = f'{camera_name}_ir1_frame'
        self.ir2_frame_name = f'{camera_name}_ir2_frame'
        self.color_frame_name = f'{camera_name}_color_frame'
        if oir1_T_imu is not None:
            self.imu_frame_name = f'{camera_name}_imu_frame'
        self.ir1_optical_frame_name = f'{camera_name}_ir1_optical_frame'
        self.ir2_optical_frame_name = f'{camera_name}_ir2_optical_frame'
        self.color_optical_frame_name = f'{camera_name}_color_optical_frame'

        self.oir1_T_imu = oir1_T_imu
        self.oir1_T_oir2 = oir1_T_oir2
        self.oir1_T_ocolor = oir1_T_ocolor

        self.ros_T_optic = np.array([[0, 0, 1, 0],
                                [-1,0, 0, 0],
                                [0, -1, 0, 0],
                                [0, 0, 0, 1]]).astype(np.float64)

        # self.broadcaster = tf2_ros.TransformBroadcaster(self)
        self.broadcaster = tf2_ros.StaticTransformBroadcaster(self)
        self.timer = self.create_timer(1, self.timer_callback)

    def timer_callback(self):
        body_T_ir1 = np.eye(4)
        body_T_ir2 = self.ros_T_optic @ self.oir1_T_oir2 @ np.linalg.inv(self.ros_T_optic)
        body_T_imu = self.ros_T_optic @ self.oir1_T_imu

        tf1 = self.makeTransformStamped(self.body_frame_name, self.ir1_frame_name, body_T_ir1)
        tf2 = self.makeTransformStamped(self.body_frame_name, self.ir2_frame_name, body_T_ir2)
        if self.oir1_T_imu is not None:
            tf3 = self.makeTransformStamped(self.body_frame_name, self.imu_frame_name, body_T_imu)
        tf4 = self.makeTransformStamped(self.ir1_frame_name, self.ir1_optical_frame_name, self.ros_T_optic)
        tf5 = self.makeTransformStamped(self.ir2_frame_name, self.ir2_optical_frame_name, self.ros_T_optic)
        self.broadcaster = tf2_ros.StaticTransformBroadcaster(self)
        self.broadcaster.sendTransform(tf1)
        self.broadcaster.sendTransform(tf2)
        if self.oir1_T_imu is not None:
            self.broadcaster.sendTransform(tf3)
        self.broadcaster.sendTransform(tf4)
        self.broadcaster.sendTransform(tf5)
        

    def makeTransformStamped(self, parent_frame, child_frame, T):
        rotation = R.from_matrix(T[:3, :3]).as_quat()
        translation = T[:3, 3]
        static_transform_stamped = TransformStamped()
        static_transform_stamped.header.stamp = self.get_clock().now().to_msg()
        static_transform_stamped.header.frame_id = parent_frame
        static_transform_stamped.child_frame_id = child_frame
        static_transform_stamped.transform.translation.x = translation[0]
        static_transform_stamped.transform.translation.y = translation[1]
        static_transform_stamped.transform.translation.z = translation[2]
        static_transform_stamped.transform.rotation.x = rotation[0]
        static_transform_stamped.transform.rotation.y = rotation[1]
        static_transform_stamped.transform.rotation.z = rotation[2]
        static_transform_stamped.transform.rotation.w = rotation[3]
        return static_transform_stamped

pygame 2.5.2 (SDL 2.28.2, Python 3.8.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [9]:
class B1ExtrinsicsBroadcaster(Node):

    def __init__(self, robot_name='b1'):
        super().__init__(f'{robot_name}_tf2_broadcaster')

        self.frames = []
        # self.broadcaster = tf2_ros.TransformBroadcaster(self)
        self.broadcaster = tf2_ros.StaticTransformBroadcaster(self)
        self.timer = self.create_timer(0.5, self.timer_callback)
        self.body_frame_name = f'{robot_name}_base_link'

    def timer_callback(self):
        for frame in self.frames:
            self.broadcaster.sendTransform(frame)

    def registerFrame(self, sensor_name, body_T_sensor):
        frame = self.makeTransformStamped(self.body_frame_name, sensor_name, body_T_sensor)
        self.frames.append(frame)

    def makeTransformStamped(self, parent_frame, child_frame, T):
        rotation = R.from_matrix(T[:3, :3]).as_quat()
        translation = T[:3, 3]
        static_transform_stamped = TransformStamped()
        static_transform_stamped.header.stamp = self.get_clock().now().to_msg()
        static_transform_stamped.header.frame_id = parent_frame
        static_transform_stamped.child_frame_id = child_frame
        static_transform_stamped.transform.translation.x = translation[0]
        static_transform_stamped.transform.translation.y = translation[1]
        static_transform_stamped.transform.translation.z = translation[2]
        static_transform_stamped.transform.rotation.x = rotation[0]
        static_transform_stamped.transform.rotation.y = rotation[1]
        static_transform_stamped.transform.rotation.z = rotation[2]
        static_transform_stamped.transform.rotation.w = rotation[3]
        return static_transform_stamped

In [10]:
rs1_broadcaster = RealsenseExtrinsicsBroadcaster('realsense', oir1_T_oir2, oir1_T_ocolor, oir1_T_imu)
rs2_broadcaster = RealsenseExtrinsicsBroadcaster('realsense2', oir1_T_oir2, oir1_T_ocolor, oir1_T_imu)

b1_broadcaster = B1ExtrinsicsBroadcaster()

T1 = np.eye(4)
T1[0,-1] = 2.
T2 = np.eye(4)
T2[0,-1] = -2.
b1_broadcaster.registerFrame(rs1_broadcaster.body_frame_name, T1)
b1_broadcaster.registerFrame(rs2_broadcaster.body_frame_name, T2)

In [11]:
excution_manager = ROS2ExecutorManager()    
excution_manager.add_node(rs1_broadcaster)
excution_manager.add_node(rs2_broadcaster)
excution_manager.add_node(b1_broadcaster)
excution_manager.start()